In [1]:
import cv2
import numpy as np

In [2]:
# 实现图像拼接的主要功能
class Stitcher:
    
    def stitch(self, images, ratio=0.75, reprojThresh=4.0, showMatches=False):
        # 输入图像对
        (img1, img2) = images
        # 通过detectAndDescribe检测A、B图片的SIFT关键特征点
        (kps_first, features_first) = self.detectAndDescribe(img1)
        (kps_second, features_second) = self.detectAndDescribe(img2)

        # 匹配两张图片的全部特征点，返回匹配结果
        M = self.matchKeypoints(kps_first, kps_second, features_first, features_second, ratio, reprojThresh)

        # 如果没有匹配成功的特征点，则退出算法
        if M is None:
            return None

        # 否则，提取匹配的结果
        (matches, H, status) = M
        # 将img1进行warpPerspective视角变换
        result = cv2.warpPerspective(img1, H, (img1.shape[1] + img2.shape[1], img1.shape[0]))

        # 对得到的result进行图像融合
        for r in range(result.shape[0]):
            left = 0
            for c in range(result.shape[1] // 2):
                if result[r, c].any():  
                    if left == 0:
                        left = c
                    alph1 = (c - left) / (result.shape[1] // 2 - left)
                    result[r, c] = img2[r, c] * (1 - alph1) + result[r, c] * alph1
                else:
                    result[r, c] = img2[r, c]

        # 检测是否需要显示图片匹配
        if showMatches:
            # 生成匹配图片
            vis = self.drawMatches(img1, img2, kps_first, kps_second, matches, status)
            # 返回结果
            return (result, vis)

        # 返回匹配结果
        return result

    def detectAndDescribe(self, image):
        # 将彩色的image转换成灰度图gray
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 建立SIFT生成器
        descriptor = cv2.SIFT_create()
        # 检测SIFT特征点
        (kps, features) = descriptor.detectAndCompute(gray, None)

        # 将结果转换成NumPy数组
        kps = np.float32([kp.pt for kp in kps])

        # 返回特征点集，及对应的描述特征
        return kps, features
    
    #匹配特征点并计算视角变换矩阵
    def matchKeypoints(self, kps_first, kps_second, features_first, features_second, ratio, reprojThresh):
        # 建立暴力匹配器
        matcher = cv2.DescriptorMatcher_create("BruteForce")

        # 使用KNN检测来自img1、img2的SIFT特征匹配对，K=2
        rawMatches = matcher.knnMatch(features_first, features_second, 2)

        matches = []
        for m in rawMatches:
            # 当最近距离跟次近距离的比值小于ratio值时，保留此匹配对
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                # 存储两个点在features_first, features_second中的索引值
                matches.append((m[0].trainIdx, m[0].queryIdx))

        # 当筛选后的匹配对大于4时，计算视角变换矩阵
        if len(matches) > 4:
            # 获取匹配对的点坐标
            ptsA = np.float32([kps_first[i] for (_, i) in matches])
            ptsB = np.float32([kps_second[i] for (i, _) in matches])

            # 计算视角变换矩阵
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

            # 返回结果
            return (matches, H, status)

        # 如果匹配对小于4时，返回None
        return None

    def drawMatches(self, img1, img2, kps_first, kps_second, matches, status):
        # 初始化可视化图片，将A、B图左右连接到一起
        (h1, w1) = img1.shape[:2]
        (h2, w2) = img2.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), dtype="uint8")
        vis[0:h1, 0:w1] = img1
        vis[0:h2, w1:] = img2

        # 联合遍历，画出匹配对
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # 当点对匹配成功时，画到可视化图上
            if s == 1:
                # 画出匹配对
                ptA = (int(kps_first[queryIdx][0]), int(kps_first[queryIdx][1]))
                ptB = (int(kps_second[trainIdx][0]) + w1, int(kps_second[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        # 返回可视化结果
        return vis

In [ ]:
if __name__ == '__main__':

    # 读取需要拼接的图像对
    img1 = cv2.imread(r"C:\Users\28106\Desktop\input\4r.jpg")
    img2 = cv2.imread(r"C:\Users\28106\Desktop\input\4l.jpg")

    # 将输入图像对拼接成全景图
    stitcher = Stitcher()
    (result, vis) = stitcher.stitch([img1, img2], showMatches=True)
    cv2.imwrite(r"C:\Users\28106\Desktop\output\result4.jpg", result)
    cv2.imshow("Result",result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()